In [1]:
import numpy as np
import os
from os.path import join
import nibabel as nib

In [2]:
#path setting
work_dir = '/nfs/z1/userhome/zzl-xsk/Featurespace'
pca_path = join(work_dir , 'prep/image_activations/pca/vectors')
activation_path = join(work_dir , 'prep/image_activations/concate-activs')
savepath = join(work_dir, 'prep/image_activations/pca-shuffle')
os.makedirs(savepath, exist_ok=True)
subs = [f'sub-{isub:02d}' for isub in range(1, 10)]

In [5]:
pca = np.squeeze(np.load(join(pca_path , 'nodstimeigenvectors-64.npy'),mmap_mode='r'))
activation = np.load(join(activation_path, 'all-sub_googlenet-conv2_activation.npy'), mmap_mode='r')
coco_activation = np.load(join(work_dir, 'prep/image_activations/coco_googlenet-conv2.npy'), mmap_mode='r')[:, 0:63, :, :]
mean = np.load(join(activation_path, 'all-sub_googlenet-conv2_63-activation_mean.npy'), mmap_mode='r').reshape(1, 63, 1, 1)
std = np.load(join(activation_path, 'all-sub_googlenet-conv2_63-activation_std.npy'), mmap_mode='r').reshape(1, 63, 1, 1)

shuffle_pca = np.copy(pca)
for i in range(pca.shape[1]):
    np.random.shuffle(shuffle_pca[:, i])

In [5]:
for isub, sub in enumerate(subs):
    print(f'{sub}')
    start, end = 4000 * isub , 4000 * (isub + 1)
    sub_activations = activation[start : end, 0 : 63, :, :]
    sub_activations = (sub_activations - mean) / std
    sub_activations = sub_activations.reshape(4000, 63, 57 * 57)

    pca_results = np.dot(shuffle_pca , sub_activations)
    pca_results = pca_results.reshape(63, 4000, 57, 57).transpose(1, 0, 2, 3)

    np.save(join(savepath, f'{sub}_googlenet-conv2_pca-shuffle-comps.npy'), pca_results)

sub-01
sub-02
sub-03
sub-04
sub-05
sub-06
sub-07
sub-08


In [7]:
coco_mean = coco_activation.mean(axis=(0, 2 , 3), keepdims=True)
coco_std = coco_activation.std(axis=(0, 2, 3), keepdims=True)

coco_activation = (coco_activation - coco_mean) / coco_std
coco_activation = coco_activation.reshape(120, 63, 57 * 57)

pca_results = np.dot(shuffle_pca , coco_activation)
pca_results = pca_results.reshape(63, 120, 57, 57).transpose(1, 0, 2, 3)
np.save(join(savepath, f'coco_googlenet-conv2_pca-shuffle-comps.npy'), pca_results)

In [8]:
pca_results.shape

(120, 63, 57, 57)